In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from finance_byu.summarize import summary
import pyarrow
# !pip uninstall pandas
# !pip install pandas==2.2.3
print(pd.__version__)

load_dotenv()  # reads .env file
SAVE_PATH = os.getenv("SAVE_PATH")

2.2.3


In [2]:
df = pd.read_feather(SAVE_PATH)

df = df[(df.date >= pd.to_datetime('1970-01-01'))]

# df = df.loc[~df['cash'].isna()]
# df = df.loc[~df['current'].isna()]
# df['ret'] = df['ret']/100

print(df)
df.describe()

         permno       date     cusip ticker  shrcd  excd   siccd        prc  \
0         10000 1986-02-28  68391610  OMFGA   10.0   3.0  3990.0   -3.25000   
1         10000 1986-03-31  68391610  OMFGA   10.0   3.0  3990.0   -4.43750   
2         10000 1986-04-30  68391610  OMFGA   10.0   3.0  3990.0   -4.00000   
3         10000 1986-05-30  68391610  OMFGA   10.0   3.0  3990.0   -3.10938   
4         10000 1986-06-30  68391610  OMFGA   10.0   3.0  3990.0   -3.09375   
...         ...        ...       ...    ...    ...   ...     ...        ...   
4963798   93436 2024-08-30  88160R10   TSLA   11.0   3.0  9999.0  214.11000   
4963799   93436 2024-09-30  88160R10   TSLA   11.0   3.0  9999.0  261.63000   
4963800   93436 2024-10-31  88160R10   TSLA   11.0   3.0  9999.0  249.85001   
4963801   93436 2024-11-29  88160R10   TSLA   11.0   3.0  9999.0  345.16000   
4963802   93436 2024-12-31  88160R10   TSLA   11.0   3.0  9999.0  403.84000   

              ret         vol  ...  fqtr  popsrc   

,permno,date,shrcd,excd,siccd,prc,ret,vol,shr,cumfacshr,GVKEY,fyearq,fqtr,current,assets,cash,mkvaltq
count,4.393271e+06,4393271,4.393271e+06,4.393271e+06,4.393271e+06,4.393271e+06,4.393271e+06,4.095726e+06,4.393271e+06,4.393271e+06,771716.000000,771716.000000,771716.000000,2.513126e+06,3.004325e+06,1.013161e+06,2.411260e+05
mean,5.621715e+04,2000-04-25 18:42:13.878197120,1.905602e+01,2.332881e+00,5.132945e+03,3.487610e+01,9.860049e-03,1.140549e+05,5.523913e+04,2.013694e+00,43922.336861,2001.669837,2.498897,1.059314e+03,8.024711e+03,6.311959e+02,6.438926e+03
min,1.000000e+04,1970-01-30 00:00:00,1.000000e+01,1.000000e+00,0.000000e+00,-1.832500e+03,-9.958300e-01,0.000000e+00,0.000000e+00,0.000000e+00,1001.000000,1980.000000,1.000000,-6.000000e-02,-6.333300e+01,-9.174000e+00,0.000000e+00
25%,2.442400e+04,1988-04-29 00:00:00,1.100000e+01,1.000000e+00,3.561000e+03,2.690000e+00,-5.913100e-02,1.291000e+03,3.471000e+03,1.000000e+00,8475.000000,1992.000000,2.000000,1.882700e+01,4.985000e+01,1.193300e+01,1.261875e+02
50%,6.315900e+04,2000-02-29 00:00:00,1.100000e+01,3.000000e+00,5.411000e+03,1.205000e+01,0.000000e+00,7.244000e+03,1.087500e+04,1.000000e+00,17140.000000,2000.000000,3.000000,8.667900e+01,2.814660e+02,5.626300e+01,5.833356e+02
75%,8.258700e+04,2013-05-31 00:00:00,1.200000e+01,3.000000e+00,6.726000e+03,2.625000e+01,6.229600e-02,4.348500e+04,3.560050e+04,1.000000e+00,61519.000000,2011.000000,3.000000,3.987480e+02,1.653510e+03,2.340000e+02,2.660050e+03
max,9.343600e+04,2024-12-31 00:00:00,7.500000e+01,3.300000e+01,9.999000e+03,7.240400e+05,3.900000e+01,2.012427e+08,2.920640e+07,1.536000e+03,356128.000000,2025.000000,4.000000,3.438409e+05,4.210048e+06,3.141480e+05,3.766500e+06
std,2.899081e+04,NaN,1.908670e+01,1.036573e+00,2.261525e+03,2.170487e+03,1.779610e-01,7.804323e+05,2.599302e+05,8.816516e+00,57460.374378,12.059066,1.114667,5.519996e+03,7.679843e+04,4.168667e+03,4.459374e+04


In [3]:
df['logret'] = np.log(1 + df['ret'])
df['mom'] = df.groupby('permno')['logret'].rolling(11).sum().reset_index(drop=True).shift()
df['me'] = df['shr'] * df['prc']


In [4]:
df['prclag'] = df.groupby('permno')['prc'].shift()
df['melag'] = df.groupby('permno')['me'].shift()
df['momlag'] = df.groupby('permno')['mom'].shift()
# df['cashtoasset'] = df["cash"] / df["assets"]
df['currenttoassetlag'] = (df["current"] / df["assets"]).shift()
df.drop(columns=["cash", "current", "assets"])
# df = df.loc[df.date >= pd.to_datetime("1990-01-01")]

df = df.query("momlag == momlag and prclag >= 5 and melag >= 100").reset_index(drop=True)

In [5]:
df.describe()

,permno,date,shrcd,excd,siccd,prc,ret,vol,shr,cumfacshr,...,assets,cash,mkvaltq,logret,mom,me,prclag,melag,momlag,currenttoassetlag
count,2.307822e+06,2307822,2.307822e+06,2.307822e+06,2.307822e+06,2.307822e+06,2.307822e+06,2.307651e+06,2.307822e+06,2.307822e+06,...,1.670957e+06,553402.000000,1.287960e+05,2.307822e+06,2.287416e+06,2.307822e+06,2.307822e+06,2.307822e+06,2.307822e+06,1.360351e+06
mean,5.079184e+04,2001-03-28 13:08:59.208655616,1.888275e+01,2.073480e+00,5.226898e+03,5.087437e+01,8.870285e-03,1.245082e+05,7.162483e+04,2.158050e+00,...,1.024613e+04,795.138137,9.088980e+03,9.299257e-04,-1.870237e-02,3.304321e+06,5.168468e+01,3.283543e+06,-2.013717e-02,4.780134e-01
min,1.000100e+04,1970-02-27 00:00:00,1.000000e+01,1.000000e+00,0.000000e+00,-1.500000e+03,-9.812950e-01,0.000000e+00,1.000000e+00,0.000000e+00,...,0.000000e+00,-9.174000,0.000000e+00,-3.978964e+00,-1.217856e+01,-1.300461e+08,5.000000e+00,1.000000e+02,-1.217856e+01,0.000000e+00
25%,2.110800e+04,1990-09-28 00:00:00,1.100000e+01,1.000000e+00,3.572000e+03,1.074000e+01,-4.808700e-02,1.837000e+03,5.180000e+03,1.000000e+00,...,1.437080e+02,21.100000,3.580650e+02,-4.928163e-02,-2.107850e-01,7.250381e+04,1.125000e+01,7.498034e+04,-2.125613e-01,2.658467e-01
50%,5.293500e+04,2000-06-30 00:00:00,1.100000e+01,2.000000e+00,5.411000e+03,1.900000e+01,5.236000e-03,9.270000e+03,1.474400e+04,1.000000e+00,...,6.143680e+02,91.391500,1.261440e+03,5.222340e-03,4.530049e-02,2.661434e+05,1.947500e+01,2.665118e+05,4.445173e-02,4.813917e-01
75%,7.867100e+04,2014-05-30 00:00:00,1.200000e+01,3.000000e+00,6.726000e+03,3.300000e+01,5.970200e-02,5.301000e+04,4.520700e+04,1.250000e+00,...,2.828726e+03,340.000000,4.434150e+03,5.798774e-02,2.502955e-01,1.134713e+06,3.320000e+01,1.127230e+06,2.497205e-01,6.808812e-01
max,8.866800e+04,2024-12-31 00:00:00,7.500000e+01,3.300000e+01,9.999000e+03,7.240400e+05,1.988359e+01,1.707594e+08,2.456884e+07,1.536000e+03,...,4.210048e+06,314148.000000,3.766500e+06,3.038964e+00,6.081498e+00,3.785304e+09,7.240400e+05,3.587438e+09,6.081498e+00,6.319211e+00
std,2.782053e+04,NaN,1.880530e+01,1.088986e+00,2.101405e+03,2.529374e+03,1.291451e-01,7.392674e+05,3.146404e+05,9.306216e+00,...,8.437227e+04,4753.785216,5.434303e+04,1.276106e-01,5.407519e-01,2.632883e+07,2.504607e+03,2.594686e+07,5.421191e-01,2.572302e-01


In [6]:
# df.loc[(df['tic']=='AAPL')&(~df['cash'].isna().index)]

In [7]:
# df = df.loc[~df['cashtoasset'].isna()]
# df = df.loc[~df['currenttoassetlag'].isna()]
df["mom_first"] = df.groupby('date')['momlag'].transform(pd.qcut,5,labels=[str(k) for k in range(1, 6)])
# df["cashtoasset_first"] = df.groupby('date')['cashtoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"]),shift()
# df["currenttoasset_first"] = df.groupby('date')['currenttoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"]).shift()
# df["cashtoasset_second"] = df.groupby(['date', 'mom_first'])['cashtoasset'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"]).shift()
dfca = df.loc[~df['currenttoassetlag'].isna()]
dfca["currenttoasset_second"] = dfca.groupby(['date', 'mom_first'])['currenttoassetlag'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"])

/var/folders/2f/49s10q117cg1smy0rrygm53h0000gn/T/ipykernel_82632/400188142.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfca["currenttoasset_second"] = dfca.groupby(['date', 'mom_first'])['currenttoassetlag'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"])
/var/folders/2f/49s10q117cg1smy0rrygm53h0000gn/T/ipykernel_82632/400188142.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfca["currenttoasset_second"] = dfca.groupby(['date', 'mom_first'])['currenttoassetlag'].transform(pd.qcut,5,labels=["a", "b", "c", "d", "e"])


In [12]:
#Equal weighted, cash to asset ratio
port = dfca.groupby(['date','mom_first','currenttoasset_second'])['ret'].mean()
port = port.unstack(level=['mom_first','currenttoasset_second'])
print('Summary statistics from month to month, mom-currentasset:')
port['spread'] = port[('5', 'e')] - port[('1', 'a')]
summary(port).round(4)

/var/folders/2f/49s10q117cg1smy0rrygm53h0000gn/T/ipykernel_82632/2369912946.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  port = dfca.groupby(['date','mom_first','currenttoasset_second'])['ret'].mean()


Summary statistics from month to month, mom-currentasset:


mom_first                     1                                          \
currenttoasset_second         a         b         c         d         e   
count                  534.0000  534.0000  534.0000  534.0000  534.0000   
mean                     0.0081    0.0099    0.0105    0.0115    0.0103   
std                      0.0475    0.0539    0.0565    0.0616    0.0702   
tstat                    3.9595    4.2575    4.3035    4.2982    3.3941   
pval                     0.0001    0.0000    0.0000    0.0000    0.0007   
min                     -0.2331   -0.3111   -0.3110   -0.3184   -0.3302   
25%                     -0.0157   -0.0183   -0.0217   -0.0240   -0.0329   
50%                      0.0131    0.0136    0.0150    0.0162    0.0129   
75%                      0.0363    0.0438    0.0429    0.0472    0.0512   
max                      0.2029    0.1768    0.2063    0.2195    0.2506   

mom_first                     2                                          ...  \
currenttoasset_second         a         b         c         d         e  ...   
count                  534.0000  534.0000  534.0000  534.0000  534.0000  ...   
mean                     0.0085    0.0089    0.0107    0.0116    0.0089  ...   
std                      0.0472    0.0533    0.0562    0.0604    0.0716  ...   
tstat                    4.1836    3.8447    4.3786    4.4429    2.8848  ...   
pval                     0.0000    0.0001    0.0000    0.0000    0.0041  ...   
min                     -0.2789   -0.3104   -0.2905   -0.3085   -0.3320  ...   
25%                     -0.0165   -0.0209   -0.0217   -0.0224   -0.0321  ...   
50%                      0.0131    0.0129    0.0134    0.0161    0.0129  ...   
75%                      0.0367    0.0424    0.0432    0.0491    0.0538  ...   
max                      0.1728    0.1835    0.2094    0.1872    0.2503  ...   

mom_first                     4                                       5  \
currenttoasset_second         b         c         d         e         a   
count                  534.0000  534.0000  534.0000  534.0000  534.0000   
mean                     0.0084    0.0116    0.0098    0.0095    0.0094   
std                      0.0534    0.0559    0.0608    0.0725    0.0483   
tstat                    3.6459    4.8075    3.7266    3.0160    4.5058   
pval                     0.0003    0.0000    0.0002    0.0027    0.0000   
min                     -0.2963   -0.3007   -0.3056   -0.3320   -0.2543   
25%                     -0.0223   -0.0167   -0.0254   -0.0343   -0.0139   
50%                      0.0111    0.0164    0.0144    0.0133    0.0128   
75%                      0.0426    0.0468    0.0467    0.0521    0.0330   
max                      0.1934    0.1838    0.2201    0.3105    0.3086   

mom_first                                                        spread  
currenttoasset_second         b         c         d         e            
count                  534.0000  534.0000  534.0000  534.0000  534.0000  
mean                     0.0095    0.0106    0.0102    0.0106    0.0025  
std                      0.0517    0.0565    0.0604    0.0705    0.0455  
tstat                    4.2571    4.3177    3.9003    3.4853    1.2642  
pval                     0.0000    0.0000    0.0001    0.0005    0.2067  
min                     -0.2773   -0.2981   -0.2999   -0.3235   -0.1728  
25%                     -0.0165   -0.0214   -0.0240   -0.0311   -0.0231  
50%                      0.0139    0.0146    0.0121    0.0156   -0.0001  
75%                      0.0395    0.0453    0.0487    0.0514    0.0284  
max                      0.1932    0.1829    0.2051    0.2973    0.2557  

[10 rows x 26 columns]

In [9]:
# Equal weighted, cash to asset ratio
port = df.groupby(['date','mom_first'])['ret'].mean()
port = port.unstack(level=['mom_first'])
port['spread'] = port['5'] - port['1']
print('Summary statistics from month to month, mom:')
summary(port).round(4)

/var/folders/2f/49s10q117cg1smy0rrygm53h0000gn/T/ipykernel_82632/1986546384.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  port = df.groupby(['date','mom_first'])['ret'].mean()


Summary statistics from month to month, mom:


mom_first,1,2,3,4,5,spread
count,659.0000,659.0000,659.0000,659.0000,659.0000,659.0000
mean,0.0099,0.0099,0.0097,0.0097,0.0100,0.0001
std,0.0518,0.0522,0.0526,0.0521,0.0518,0.0064
tstat,4.9058,4.8698,4.7181,4.7611,4.9537,0.3621
pval,0.0000,0.0000,0.0000,0.0000,0.0000,0.7174
min,-0.2742,-0.2736,-0.2785,-0.2726,-0.2641,-0.0243
25%,-0.0184,-0.0217,-0.0199,-0.0193,-0.0184,-0.0039
50%,0.0128,0.0127,0.0115,0.0114,0.0123,-0.0001
75%,0.0417,0.0426,0.0422,0.0419,0.0402,0.0039
max,0.2345,0.2284,0.2192,0.2437,0.2291,0.0283


In [10]:
# Value weighted:
df['weights'] = df.groupby(['date','mom_first'])['melag'].transform(lambda x: x / x.sum())
df['valueReturns'] = df['ret'] * df['weights']
port = df.groupby(['date','mom_first'])['valueReturns'].sum()
port = port.unstack(level=['mom_first'])
port['spread'] = port['5'] - port['1']
print('Summary statistics from month to month, mom:')
summary(port).round(4)

/var/folders/2f/49s10q117cg1smy0rrygm53h0000gn/T/ipykernel_82632/3822269860.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['weights'] = df.groupby(['date','mom_first'])['melag'].transform(lambda x: x / x.sum())


Summary statistics from month to month, mom:


/var/folders/2f/49s10q117cg1smy0rrygm53h0000gn/T/ipykernel_82632/3822269860.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  port = df.groupby(['date','mom_first'])['valueReturns'].sum()


mom_first,1,2,3,4,5,spread
count,659.0000,659.0000,659.0000,659.0000,659.0000,659.0000
mean,0.0090,0.0095,0.0094,0.0095,0.0096,0.0006
std,0.0475,0.0454,0.0458,0.0467,0.0464,0.0148
tstat,4.8455,5.3753,5.2748,5.2245,5.3005,1.0665
pval,0.0000,0.0000,0.0000,0.0000,0.0000,0.2866
min,-0.2389,-0.2330,-0.2151,-0.2193,-0.2259,-0.0627
25%,-0.0176,-0.0165,-0.0156,-0.0162,-0.0195,-0.0071
50%,0.0125,0.0122,0.0138,0.0123,0.0139,0.0007
75%,0.0379,0.0374,0.0386,0.0384,0.0398,0.0092
max,0.1580,0.2140,0.1647,0.1778,0.1665,0.0778
